In [ ]:
!pip install mysql-connector-python
import mysql.connector
import random
from datetime import datetime, timedelta

# ---------- DB Connection ----------
con = mysql.connector.connect(
    host="127.0.0.1", user="root", password="", database="smartshow_db", port=3306
)
cursor = con.cursor()
print("✅ Connected to MySQL")

# ---------- Validation ----------
def valid_email(email):
    return email.endswith("@gmail.com")

def valid_password(pw):
    return any(c.isupper() for c in pw) and any(c.islower() for c in pw) and "@" in pw

def generate_otp():
    return str(random.randint(100000, 999999))

# ---------- Register ----------
def register_user():
    name = input("Enter name: ")
    email = input("Enter email: ")
    if not valid_email(email):
        print("❌ Email must be @gmail.com")
        return

    # ✅ Check unique email
    cursor.execute("SELECT email FROM users WHERE email=%s", (email,))
    if cursor.fetchone():
        print("❌ Email already exists!")
        return

    pw = input("Enter password: ")
    if not valid_password(pw):
        print("❌ Password must have 1 uppercase, 1 lowercase & 1 '@'")
        return

    # ✅ Check unique password
    cursor.execute("SELECT password FROM users WHERE password=%s", (pw,))
    if cursor.fetchone():
        print("❌ Password already used! Choose another")
        return

    otp = generate_otp()
    expiry = datetime.now() + timedelta(minutes=2)

    # ✅ Insert without specifying ID (auto-increment line-wise)
    cursor.execute(
        "INSERT INTO users (name,email,password,otp,otp_expiry) VALUES (%s,%s,%s,%s,%s)",
        (name,email,pw,otp,expiry)
    )
    con.commit()
    print(f"🔐 Your OTP: {otp} (valid 2 min)")

    user_otp = input("Enter OTP: ")
    if user_otp != otp:
        print("❌ Invalid OTP")
        return

    # Clear OTP
    cursor.execute("UPDATE users SET otp=NULL,otp_expiry=NULL WHERE email=%s",(email,))
    con.commit()
    print("✅ Registered successfully")

# ---------- Login ----------
def login_user():
    email = input("Enter email: ")
    pw = input("Enter password: ")
    cursor.execute("SELECT * FROM users WHERE email=%s AND password=%s",(email,pw))
    if cursor.fetchone():
        print("✅ Login successful")
        return True
    print("❌ Invalid email or password")
    return False

# ---------- Main ----------
while True:
    print("\n1. Register  2. Login  3. Exit")
    choice = input("Choose option: ")

    if choice=="1":
        register_user()
    elif choice=="2":
        login_user()
    elif choice=="3":
        print("👋 Goodbye")
        break
    else:
        print("❌ Invalid choice")


✅ Connected to MySQL

1. Register  2. Login  3. Exit


In [ ]:
!pip install mysql-connector-python
import mysql.connector
import sqlite3
import random
import json
from datetime import datetime, timedelta

# ---------- MySQL DB Connection ----------
mysql_conn = mysql.connector.connect(
    host="127.0.0.1", user="root", password="", database="smartshow_db", port=3306
)
mysql_cursor = mysql_conn.cursor()
print("✅ Connected to MySQL")

# ---------- SQLite Connection (Movies) ----------
sqlite_conn = sqlite3.connect("movies")
sqlite_cursor = sqlite_conn.cursor()

# ---------- Validation ----------
def valid_email(email):
    return email.endswith("@gmail.com")

def valid_password(pw):
    return any(c.isupper() for c in pw) and any(c.islower() for c in pw) and "@" in pw

def generate_otp():
    return str(random.randint(100000, 999999))

# ---------- Register ----------
def register_user():
    name = input("Enter name: ")

    while True:
        email = input("Enter email: ")
        if not valid_email(email):
            print("❌ Email must be @gmail.com")
            continue
        mysql_cursor.execute("SELECT email FROM users WHERE email=%s", (email,))
        if mysql_cursor.fetchone():
            print("❌ Email already exists!")
            continue
        break

    while True:
        pw = input("Enter password: ")
        if not valid_password(pw):
            print("❌ Password must have 1 uppercase, 1 lowercase & 1 '@'")
            continue
        mysql_cursor.execute("SELECT password FROM users WHERE password=%s", (pw,))
        if mysql_cursor.fetchone():
            print("❌ Password already used! Choose another")
            continue
        break

    otp = generate_otp()
    expiry = datetime.now() + timedelta(minutes=2)
    mysql_cursor.execute(
        "INSERT INTO users (name,email,password,otp,otp_expiry) VALUES (%s,%s,%s,%s,%s)",
        (name,email,pw,otp,expiry)
    )
    mysql_conn.commit()
    print(f"🔐 Your OTP: {otp} (Valid for 2 minutes)")

    while True:
        user_otp = input("Enter OTP: ")
        mysql_cursor.execute("SELECT otp, otp_expiry FROM users WHERE email=%s",(email,))
        db_otp, db_expiry = mysql_cursor.fetchone()

        if datetime.now() > db_expiry:
            otp = generate_otp()
            expiry = datetime.now() + timedelta(minutes=2)
            mysql_cursor.execute(
                "UPDATE users SET otp=%s, otp_expiry=%s WHERE email=%s",
                (otp, expiry, email)
            )
            mysql_conn.commit()
            print(f"⏳ OTP expired! New OTP: {otp} (Valid for 2 minutes)")
            continue

        if user_otp != db_otp:
            print("❌ Invalid OTP, try again.")
            continue

        mysql_cursor.execute("UPDATE users SET otp=NULL, otp_expiry=NULL WHERE email=%s",(email,))
        mysql_conn.commit()
        print("✅ Registered successfully")
        return email

# ---------- Login ----------
def login_user():
    while True:
        email = input("Enter email: ")
        pw = input("Enter password: ")
        mysql_cursor.execute("SELECT * FROM users WHERE email=%s AND password=%s",(email,pw))
        if mysql_cursor.fetchone():
            print("✅ Login successful")
            return email
        else:
            print("❌ Invalid email or password")
            retry = input("Try again? (y/n): ")
            if retry.lower() != "y":
                return None

# ---------- Theater Class ----------
class Theater:
    def __init__(self, theater_id, name, area, city, theater_type, total_screens, base_price, total_seats):
        self.theater_id = theater_id
        self.name = name
        self.area = area
        self.city = city
        self.theater_type = theater_type
        self.total_screens = total_screens
        self.base_price = base_price
        self.total_seats = total_seats
        
        # हर theater का अपना unique seat configuration
        self.seat_config = self.generate_seat_config()
        
    def generate_seat_config(self):
        """हर theater के लिए unique seat layout generate करें"""
        configs = {
            1: {"rows": ["A", "B", "C", "D", "E"], "seats_per_row": 15},
            2: {"rows": ["A", "B", "C", "D", "E", "F"], "seats_per_row": 12},
            3: {"rows": ["A", "B", "C", "D"], "seats_per_row": 20},
            4: {"rows": ["A", "B", "C", "D", "E", "F", "G"], "seats_per_row": 10},
            5: {"rows": ["A", "B", "C"], "seats_per_row": 25},
            6: {"rows": ["A", "B", "C", "D", "E"], "seats_per_row": 18},
            7: {"rows": ["A", "B", "C", "D"], "seats_per_row": 22},
            8: {"rows": ["A", "B", "C", "D", "E", "F"], "seats_per_row": 16},
            9: {"rows": ["A", "B", "C"], "seats_per_row": 30},
            10: {"rows": ["A", "B", "C", "D", "E"], "seats_per_row": 14}
        }
        
        # Theater ID के based पर configuration select करें
        config_id = (self.theater_id % 10) + 1
        return configs.get(config_id, {"rows": ["A", "B", "C", "D", "E"], "seats_per_row": 10})

    def show_details(self):
        print("\n--- Theatre Details ---")
        print(f"ID         : {self.theater_id}")
        print(f"Name       : {self.name}")
        print(f"Area       : {self.area}")
        print(f"City       : {self.city}")
        print(f"Type       : {self.theater_type}")
        print(f"Screens    : {self.total_screens}")
        print(f"Base Price : ₹{self.base_price}")
        print(f"Total Seats: {self.total_seats}")
        print(f"Seat Layout: {len(self.seat_config['rows'])} rows ({', '.join(self.seat_config['rows'])})")
        print(f"Seats/Row  : {self.seat_config['seats_per_row']}")

# ---------- Booking Class ----------
class Booking:
    def __init__(self, user_email, movie_id, movie_name, theatre_id, theatre_name, 
                 show_date, show_time, base_price, total_seats_in_theatre, seat_config):
        self.user_email = user_email
        self.movie_id = movie_id
        self.movie_name = movie_name
        self.theatre_id = theatre_id
        self.theatre_name = theatre_name
        self.show_date = show_date
        self.show_time = show_time
        self.base_price = base_price
        
        # Theater-specific seat configuration use करें
        self.rows = seat_config["rows"]
        self.total_seats_per_row = seat_config["seats_per_row"]
        
        # हर theater के लिए unique row pricing
        self.row_prices = self.generate_row_prices()
        
        self.total_seats = len(self.rows) * self.total_seats_per_row
        self.available_seats = min(self.total_seats, total_seats_in_theatre)
        self.booked_seats = 0
        self.total_amount = 0
        
        # Store booking details per row
        self.row_bookings = {row: {"booked": 0, "available": self.total_seats_per_row} for row in self.rows}
        
        # Booking details for database
        self.booking_details = []  # Store individual seat bookings
        self.seat_numbers = []  # Store seat numbers like A1, A2, etc.
        
        # Adjust based on theatre's actual capacity
        if self.total_seats > total_seats_in_theatre:
            print(f"⚠️  Adjusting seat layout to theatre capacity: {total_seats_in_theatre}")
            self.total_seats = total_seats_in_theatre
            self.available_seats = total_seats_in_theatre
            
            # Equal distribution across rows
            seats_per_row = total_seats_in_theatre // len(self.rows)
            remainder = total_seats_in_theatre % len(self.rows)
            
            for i, row in enumerate(self.rows):
                if i < remainder:
                    self.row_bookings[row]["available"] = seats_per_row + 1
                else:
                    self.row_bookings[row]["available"] = seats_per_row
    
    def generate_row_prices(self):
        """हर theater के लिए unique row pricing generate करें"""
        # Theater ID के based पर different pricing strategies
        pricing_strategies = {
            1: {"A": 1.5, "B": 1.3, "C": 1.1, "D": 0.9, "E": 0.7, "F": 0.6, "G": 0.5},
            2: {"A": 1.6, "B": 1.4, "C": 1.2, "D": 1.0, "E": 0.8, "F": 0.7},
            3: {"A": 1.4, "B": 1.2, "C": 1.0, "D": 0.8},
            4: {"A": 1.5, "B": 1.25, "C": 1.0, "D": 0.85, "E": 0.7, "F": 0.6, "G": 0.5},
            5: {"A": 1.3, "B": 1.1, "C": 0.9},
            6: {"A": 1.45, "B": 1.25, "C": 1.05, "D": 0.85, "E": 0.65},
            7: {"A": 1.35, "B": 1.15, "C": 0.95, "D": 0.75},
            8: {"A": 1.5, "B": 1.3, "C": 1.1, "D": 0.9, "E": 0.7, "F": 0.6},
            9: {"A": 1.4, "B": 1.15, "C": 0.9},
            10: {"A": 1.6, "B": 1.35, "C": 1.1, "D": 0.85, "E": 0.6}
        }
        
        # Theater ID के based पर pricing strategy select करें
        strategy_id = (self.theatre_id % 10) + 1
        strategy = pricing_strategies.get(strategy_id, {"A": 1.4, "B": 1.2, "C": 1.0, "D": 0.8, "E": 0.6})
        
        # Generate actual prices
        row_prices = {}
        for row in self.rows:
            if row in strategy:
                multiplier = strategy[row]
                row_prices[row] = int(self.base_price * multiplier)
            else:
                # Default pricing for additional rows
                row_prices[row] = int(self.base_price * 0.7)
        
        return row_prices

    def display_seats_with_prices(self):
        print("\n" + "=" * 70)
        print(f"🎟 SEAT LAYOUT FOR {self.theatre_name}")
        print("=" * 70)
        print(f"{'Row':<6} {'Available':<12} {'Price':<10} {'Status':<15}")
        print("-" * 70)
        
        for row in self.rows:
            available = self.row_bookings[row]["available"]
            price = self.row_prices[row]
            booked = self.row_bookings[row]["booked"]
            status = f"{booked}/{self.total_seats_per_row} booked"
            
            if available == 0:
                status = "❌ SOLD OUT"
            elif available <= 3:
                status = f"⚠️  {available} left"
            
            print(f"{row:<6} {available:<12} ₹{price:<9} {status:<15}")
        
        print("=" * 70)
        print(f"💡 Total available seats: {self.available_seats}/{self.total_seats}")
        print(f"🎭 Theatre: {self.theatre_name} | Movie: {self.movie_name}")
        print("=" * 70)

    def book_tickets(self):
        while True:
            self.display_seats_with_prices()
            
            try:
                qty = int(input("\n🎟 How many tickets do you want to book? "))
            except ValueError:
                print("❌ Please enter a valid number of tickets.")
                continue

            if qty <= 0:
                print("❌ Ticket count must be at least 1.")
                continue
            if qty > self.available_seats:
                print(f"❌ Sorry, only {self.available_seats} seats are available.")
                continue

            # Row-wise booking with user choice
            booked_count = 0
            total_cost = 0
            booking_details = []
            self.seat_numbers = []  # Reset seat numbers
            
            # Track which seats have been booked in each row
            row_seat_trackers = {row: 1 for row in self.rows}  # Start seat numbers from 1
            
            for i in range(qty):
                print(f"\n📝 Booking ticket {i+1} of {qty}:")
                print("Available rows with prices:")
                
                available_rows = []
                for row in self.rows:
                    available = self.row_bookings[row]["available"]
                    if available > 0:
                        price = self.row_prices[row]
                        print(f"  Row {row}: {available} seats available - ₹{price}")
                        available_rows.append(row)
                
                if not available_rows:
                    print("❌ No seats available in any row!")
                    return False
                
                while True:
                    chosen_row = input(f"Choose row for ticket {i+1} ({'/'.join(available_rows)}): ").upper()
                    if chosen_row in available_rows:
                        # Generate seat number (e.g., A1, A2, etc.)
                        seat_number = f"{chosen_row}{row_seat_trackers[chosen_row]}"
                        row_seat_trackers[chosen_row] += 1
                        
                        # Book this seat
                        self.row_bookings[chosen_row]["available"] -= 1
                        self.row_bookings[chosen_row]["booked"] += 1
                        booked_count += 1
                        price = self.row_prices[chosen_row]
                        total_cost += price
                        
                        # Store booking details
                        booking_details.append((chosen_row, price, seat_number))
                        self.seat_numbers.append(seat_number)
                        
                        print(f"✅ Booked Seat {seat_number} (Row {chosen_row}) for ₹{price}")
                        break
                    else:
                        print("❌ Invalid row or no seats available in that row. Choose again.")
            
            self.booked_seats += qty
            self.available_seats -= qty
            self.total_amount = total_cost
            self.booking_details = booking_details
            
            # Display booking summary
            print("\n" + "=" * 60)
            print("📋 BOOKING CONFIRMATION")
            print("=" * 60)
            print(f"Theatre: {self.theatre_name}")
            print(f"Movie: {self.movie_name}")
            print(f"Show: {self.show_date} at {self.show_time}")
            print("-" * 60)
            
            for i, (row, price, seat) in enumerate(booking_details, 1):
                print(f"Ticket {i}: Seat {seat} (Row {row}) - ₹{price}")
            
            print("-" * 60)
            print(f"Total Tickets: {qty}")
            print(f"Total Amount: ₹{self.total_amount}")
            print("=" * 60)
            print("✅ Booking confirmed!")
            
            return True

    def save_to_db(self):
        try:
            # First, get current theatre details
            mysql_cursor.execute("SELECT total_seats, base_price FROM theatres WHERE theater_id=%s", (self.theatre_id,))
            theatre_result = mysql_cursor.fetchone()
            
            if not theatre_result:
                print("❌ Theatre not found!")
                return False
                
            theatre_total_seats, theatre_base_price = theatre_result
            
            # Create detailed booking information
            booking_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            # Convert seat numbers to JSON array
            seat_numbers_json = json.dumps(self.seat_numbers)
            
            # Convert booking details to JSON
            booking_details_json = json.dumps([
                {
                    "seat": seat,
                    "row": row,
                    "price": price
                }
                for row, price, seat in self.booking_details
            ])
            
            # Convert row bookings to JSON
            row_bookings_json = json.dumps(self.row_bookings)
            
            # Calculate remaining seats in theatre after booking
            remaining_seats_in_theatre = theatre_total_seats - self.booked_seats
            
            # Save detailed booking information
            mysql_cursor.execute("""
            INSERT INTO bookings 
            (user_email, movie_id, movie_name, theatre_id, theatre_name,
             show_date, show_time, base_price, total_seats, available_seats,
             booked_seats, total_amount, booking_date, seat_numbers,
             booking_details, row_bookings, remaining_seats_in_theatre)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                self.user_email, 
                self.movie_id, 
                self.movie_name,
                self.theatre_id, 
                self.theatre_name,
                self.show_date, 
                self.show_time, 
                self.base_price,
                self.total_seats, 
                self.available_seats,
                self.booked_seats, 
                self.total_amount,
                booking_date,
                seat_numbers_json,
                booking_details_json,
                row_bookings_json,
                remaining_seats_in_theatre
            ))
            
            mysql_conn.commit()
            
            # Update theatre's available seats
            mysql_cursor.execute("""
            UPDATE theatres 
            SET total_seats = total_seats - %s 
            WHERE theater_id = %s
            """, (self.booked_seats, self.theatre_id))
            mysql_conn.commit()
            
            # Also update or create a theatre_seats record for detailed tracking
            mysql_cursor.execute("""
            INSERT INTO theatre_seats 
            (theatre_id, show_date, show_time, total_seats, booked_seats, available_seats, 
             seat_layout, last_updated)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE 
            booked_seats = booked_seats + VALUES(booked_seats),
            available_seats = available_seats - VALUES(booked_seats),
            seat_layout = VALUES(seat_layout),
            last_updated = VALUES(last_updated)
            """, (
                self.theatre_id,
                self.show_date,
                self.show_time,
                theatre_total_seats,
                self.booked_seats,
                remaining_seats_in_theatre,
                json.dumps(self.row_bookings),
                booking_date
            ))
            mysql_conn.commit()
            
            # Print detailed booking summary
            print("\n" + "=" * 70)
            print("📦 BOOKING DETAILS SAVED TO DATABASE")
            print("=" * 70)
            print(f"📋 Booking Summary:")
            print(f"   User: {self.user_email}")
            print(f"   Movie: {self.movie_name}")
            print(f"   Theatre: {self.theatre_name}")
            print(f"   Show: {self.show_date} at {self.show_time}")
            print(f"   Seats Booked: {', '.join(self.seat_numbers)}")
            print(f"   Total Tickets: {self.booked_seats}")
            print(f"   Total Amount: ₹{self.total_amount}")
            print(f"   Booking Time: {booking_date}")
            print(f"   Theatre ID: {self.theatre_id}")
            print(f"   Base Price: ₹{self.base_price}")
            print("-" * 70)
            print(f"📊 Theatre Status:")
            print(f"   Original Capacity: {theatre_total_seats}")
            print(f"   Now Available: {remaining_seats_in_theatre}")
            print(f"   Just Booked: {self.booked_seats}")
            print("=" * 70)
            
            # Show what was saved in database
            print("\n💾 Database Records Updated:")
            print("   1. bookings table: Complete booking details")
            print("   2. theatres table: Updated available seats")
            print("   3. theatre_seats table: Detailed seat tracking")
            
            return True
            
        except Exception as e:
            print(f"❌ Error saving booking to database: {e}")
            mysql_conn.rollback()
            return False

    def view_booking_summary(self):
        """Display a summary of the booking"""
        print("\n" + "=" * 70)
        print("🎫 BOOKING SUMMARY")
        print("=" * 70)
        print(f"Booking ID: {self.booking_id if hasattr(self, 'booking_id') else 'Not saved yet'}")
        print(f"User: {self.user_email}")
        print(f"Movie: {self.movie_name}")
        print(f"Theatre: {self.theatre_name}")
        print(f"Show Date: {self.show_date}")
        print(f"Show Time: {self.show_time}")
        print(f"Total Seats: {self.total_seats}")
        print(f"Booked Seats: {self.booked_seats}")
        print(f"Available Seats: {self.available_seats}")
        print(f"Total Amount: ₹{self.total_amount}")
        print(f"Seat Numbers: {', '.join(self.seat_numbers)}")
        print("=" * 70)

# ---------- Movie Class ----------
class Movie:
    def __init__(self, user_email):
        self.movie_theatre_map = {}
        self.user_email = user_email

    def movies(self):
        while True:
            print("\n" + "=" * 50)
            print("🎭 CHOOSE YOUR MOOD FOR MOVIES")
            print("=" * 50)
            print("1. 😍 Romantic")
            print("2. 😌 Relaxed")
            print("3. 🎉 Excited")
            print("4. 👨‍👩‍👧‍👦 Family")
            print("0. 👋 Exit")
            print("=" * 50)
            
            mood_map = {"1":"Romantic","2":"Relaxed","3":"Excited","4":"Family"}
            choice = input("\nEnter your choice (0-4): ")
            
            if choice=="0":
                print("\n👋 Thank you for using SmartShow! Hope to see you again!")
                break
                
            mood = mood_map.get(choice)
            if not mood:
                print("❌ Invalid choice! Please enter 0-4")
                continue

            sqlite_cursor.execute("SELECT id,movie_name FROM movies WHERE mood=? LIMIT 10",(mood,))
            movies = sqlite_cursor.fetchall()
            if not movies:
                print(f"❌ No {mood.lower()} movies found")
                continue

            print(f"\n" + "=" * 50)
            print(f"🎬 {mood.upper()} MOVIES")
            print("=" * 50)
            
            allowed_movie_ids = []
            for m in movies:
                print(f"{m[0]:<3}. {m[1]}")
                allowed_movie_ids.append(str(m[0]))

            movie_id = input("\n🎥 Select Movie ID: ")
            if movie_id not in allowed_movie_ids:
                print("❌ Invalid Movie ID for this mood!")
                continue

            # Get theaters showing this movie
            if movie_id not in self.movie_theatre_map:
                mysql_cursor.execute("SELECT theater_id FROM theatres WHERE total_seats > 0")
                all_ids = [row[0] for row in mysql_cursor.fetchall()]
                if len(all_ids) >= 5:
                    self.movie_theatre_map[movie_id] = random.sample(all_ids, min(5, len(all_ids)))
                else:
                    self.movie_theatre_map[movie_id] = all_ids

            fixed_ids = self.movie_theatre_map[movie_id]
            if not fixed_ids:
                print("❌ No theatres available with seats!")
                continue
                
            mysql_cursor.execute(f"SELECT * FROM theatres WHERE theater_id IN ({','.join(map(str,fixed_ids))})")
            rows = mysql_cursor.fetchall()
            
            print(f"\n" + "=" * 60)
            print(f"🎭 THEATRES SHOWING THIS MOVIE")
            print("=" * 60)
            
            theaters = []
            for r in rows:
                t = Theater(*r)
                theaters.append(t)
                print(f"{t.theater_id:>3}. {t.name:<25} {t.city:<15} ₹{t.base_price:<6} {t.total_seats:>3} seats")

            while True:
                print("\n" + "-" * 60)
                tid = input("Enter Theatre ID to book tickets (or '0' to go back): ")
                
                if tid == "0":
                    break
                    
                if tid not in [str(i) for i in fixed_ids]:
                    print("❌ Invalid Theatre ID for this movie! Please choose from the list above.")
                    continue
                    
                break
            
            if tid == "0":
                continue

            # Get selected theatre details
            mysql_cursor.execute("SELECT * FROM theatres WHERE theater_id=%s",(tid,))
            res = mysql_cursor.fetchone()
            if res:
                t = Theater(*res)
                print("\n" + "=" * 60)
                print("🎟 THEATRE DETAILS")
                print("=" * 60)
                t.show_details()
                
                # Check if theatre has seats
                if t.total_seats <= 0:
                    print("\n❌ Sorry, this theatre is fully booked!")
                    continue

                # ---------- Booking Process ----------
                movie_name_input = [m[1] for m in movies if str(m[0])==movie_id][0]
                show_date_input = datetime.now().date()
                show_time_input = "07:30 PM"

                print("\n" + "=" * 60)
                print("🎫 BOOKING PROCESS")
                print("=" * 60)
                
                booking_obj = Booking(
                    user_email=self.user_email,
                    movie_id=int(movie_id),
                    movie_name=movie_name_input,
                    theatre_id=int(tid),
                    theatre_name=t.name,
                    show_date=show_date_input,
                    show_time=show_time_input,
                    base_price=t.base_price,
                    total_seats_in_theatre=t.total_seats,
                    seat_config=t.seat_config
                )
                
                if booking_obj.book_tickets():
                    if booking_obj.save_to_db():
                        # Show booking summary
                        booking_obj.view_booking_summary()
                        
                        # Ask for another booking
                        print("\n" + "-" * 60)
                        another = input("🎫 Do you want to book more tickets? (y/n): ")
                        if another.lower() != 'y':
                            print("\n👋 Thank you for booking with SmartShow!")
                            return
            else:
                print("❌ Theatre details not found!")

# ---------- View Previous Bookings ----------
def view_my_bookings(user_email):
    print("\n" + "=" * 70)
    print("📖 MY BOOKING HISTORY")
    print("=" * 70)
    
    try:
        mysql_cursor.execute("""
        SELECT booking_id, movie_name, theatre_name, show_date, show_time, 
               booked_seats, total_amount, booking_date, seat_numbers
        FROM bookings 
        WHERE user_email = %s 
        ORDER BY booking_date DESC
        LIMIT 10
        """, (user_email,))
        
        bookings = mysql_cursor.fetchall()
        
        if not bookings:
            print("No bookings found for your account.")
            return
        
        for booking in bookings:
            print(f"\n📋 Booking ID: {booking[0]}")
            print(f"   Movie: {booking[1]}")
            print(f"   Theatre: {booking[2]}")
            print(f"   Show: {booking[3]} at {booking[4]}")
            print(f"   Tickets: {booking[5]}")
            print(f"   Amount: ₹{booking[6]}")
            print(f"   Booked on: {booking[7]}")
            
            # Parse seat numbers from JSON
            if booking[8]:
                try:
                    seat_nums = json.loads(booking[8])
                    print(f"   Seats: {', '.join(seat_nums)}")
                except:
                    print(f"   Seats: {booking[8]}")
            
            print("-" * 70)
            
    except Exception as e:
        print(f"❌ Error fetching bookings: {e}")

# ---------- Main Program ----------
def main():
    print("=" * 70)
    print("        🎬 WELCOME TO SMARTSHOW - YOUR MOVIE BOOKING PARTNER 🎬        ")
    print("=" * 70)

    logged_in_email = None
    while True:
        print("\n🔐 ACCOUNT OPTIONS")
        print("-" * 40)
        print("1. 📝 Register (New User)")
        print("2. 🔑 Login (Existing User)")
        print("3. 📖 View My Bookings")
        print("0. 👋 Exit")
        print("-" * 40)
        
        choice = input("\nChoose option: ")
        if choice == "0":
            print("\n👋 Thank you for visiting SmartShow!")
            break
        elif choice == "1":
            logged_in_email = register_user()
            if logged_in_email:
                break
        elif choice == "2":
            logged_in_email = login_user()
            if logged_in_email:
                break
        elif choice == "3":
            if logged_in_email:
                view_my_bookings(logged_in_email)
            else:
                print("❌ Please login first to view your bookings.")
        else:
            print("❌ Invalid choice")
            continue
    
    if not logged_in_email:
        return

    # Show available theatres
    print("\n" + "=" * 70)
    print("           🎭 AVAILABLE THEATRES 🎭           ")
    print("=" * 70)

    mysql_cursor.execute("SELECT * FROM theatres WHERE total_seats > 0 ORDER BY city, name LIMIT 20")
    rows = mysql_cursor.fetchall()

    if rows:
        print(f"\n{'ID':<5} {'Name':<25} {'City':<15} {'Type':<12} {'Price':<8} {'Seats':<8}")
        print("-" * 80)
        
        for row in rows:
            t = Theater(*row)
            print(f"{t.theater_id:<5} {t.name:<25} {t.city:<15} {t.theater_type:<12} ₹{t.base_price:<7} {t.total_seats:<8}")
        
        print("\n💡 Each theatre has unique seat layout and pricing!")
    else:
        print("❌ No theatres available at the moment!")

    # Movie selection & booking
    obj = Movie(logged_in_email)
    obj.movies()

    # Close DB connections
    sqlite_conn.close()
    mysql_cursor.close()
    mysql_conn.close()
    print("\n" + "=" * 70)
    print("✅ Connections closed. Have a great day! 🎬")
    print("=" * 70)

# ---------- Database Table Creation Script ----------
def create_tables():
    """Create necessary tables if they don't exist"""
    create_tables_sql = """
    -- Create bookings table with detailed columns
    CREATE TABLE IF NOT EXISTS bookingss (
        booking_id INT AUTO_INCREMENT PRIMARY KEY,
        user_email VARCHAR(100) NOT NULL,
        movie_id INT NOT NULL,
        movie_name VARCHAR(255) NOT NULL,
        theatre_id INT NOT NULL,
        theatre_name VARCHAR(255) NOT NULL,
        show_date DATE NOT NULL,
        show_time TIME NOT NULL,
        base_price INT NOT NULL,
        total_seats INT NOT NULL,
        available_seats INT NOT NULL,
        booked_seats INT NOT NULL,
        total_amount INT NOT NULL,
        booking_date DATETIME NOT NULL,
        seat_numbers TEXT,
        booking_details TEXT,
        row_bookings TEXT,
        remaining_seats_in_theatre INT,
        FOREIGN KEY (user_email) REFERENCES users(email) ON DELETE CASCADE
    );

    -- Create theatre_seats table for detailed seat tracking
    CREATE TABLE IF NOT EXISTS theatre_seats (
        id INT AUTO_INCREMENT PRIMARY KEY,
        theatre_id INT NOT NULL,
        show_date DATE NOT NULL,
        show_time TIME NOT NULL,
        total_seats INT NOT NULL,
        booked_seats INT NOT NULL DEFAULT 0,
        available_seats INT NOT NULL,
        seat_layout TEXT,
        last_updated DATETIME NOT NULL,
        UNIQUE KEY unique_show (theatre_id, show_date, show_time)
    );

    -- Add columns to theatres table if not exists
    ALTER TABLE theatres 
    ADD COLUMN IF NOT EXISTS base_price INT DEFAULT 250,
    ADD COLUMN IF NOT EXISTS total_seats INT DEFAULT 50;
    """
    
    print("✅ Database tables created/updated successfully!")

if __name__ == "__main__":
    # Uncomment the line below to create tables if needed
    # create_tables()
    main()

✅ Connected to MySQL
        🎬 WELCOME TO SMARTSHOW - YOUR MOVIE BOOKING PARTNER 🎬        

🔐 ACCOUNT OPTIONS
----------------------------------------
1. 📝 Register (New User)
2. 🔑 Login (Existing User)
3. 📖 View My Bookings
0. 👋 Exit
----------------------------------------



Choose option:  1
Enter name:  deanshs
Enter email:  jdidjij@gmail.com
Enter password:  jdiedjjA@


🔐 Your OTP: 369342 (Valid for 2 minutes)


Enter OTP:  369342


✅ Registered successfully

           🎭 AVAILABLE THEATRES 🎭           

ID    Name                      City            Type         Price    Seats   
--------------------------------------------------------------------------------
12    Cinepolis One Mall        Ahmedabad       Multiplex    ₹175     240     
3     City Gold Ashram          Ahmedabad       Multiplex    ₹140     190     
10    City Gold Bapunagar       Ahmedabad       Single Screen ₹155     210     
18    City Pulse                Ahmedabad       Multiplex    ₹135     180     
5     Drive In Cinema           Ahmedabad       Single Screen ₹180     250     
14    Fun Cinemas               Ahmedabad       Multiplex    ₹195     270     
17    Galaxy Cinema             Ahmedabad       Single Screen ₹225     320     
2     INOX AlphaOne             Ahmedabad       Multiplex    ₹150     200     
20    INOX CG Road              Ahmedabad       Multiplex    ₹155     220     
9     INOX Himalaya             Ahmedabad       Multi

...

❌ Sorry, only 70 seats are available.

🎟 SEAT LAYOUT FOR City Gold Ashram
Row    Available    Price      Status         
----------------------------------------------------------------------
A      10           ₹210       0/10 booked    
B      10           ₹175       0/10 booked    
C      10           ₹140       0/10 booked    
D      10           ₹119       0/10 booked    
E      10           ₹98        0/10 booked    
F      10           ₹84        0/10 booked    
G      10           ₹70        0/10 booked    
💡 Total available seats: 70/70
🎭 Theatre: City Gold Ashram | Movie: Our World



🎟 How many tickets do you want to book?  70



📝 Booking ticket 1 of 70:
Available rows with prices:
  Row A: 10 seats available - ₹210
  Row B: 10 seats available - ₹175
  Row C: 10 seats available - ₹140
  Row D: 10 seats available - ₹119
  Row E: 10 seats available - ₹98
  Row F: 10 seats available - ₹84
  Row G: 10 seats available - ₹70


Choose row for ticket 1 (A/B/C/D/E/F/G):  A


✅ Booked Seat A1 (Row A) for ₹210

📝 Booking ticket 2 of 70:
Available rows with prices:
  Row A: 9 seats available - ₹210
  Row B: 10 seats available - ₹175
  Row C: 10 seats available - ₹140
  Row D: 10 seats available - ₹119
  Row E: 10 seats available - ₹98
  Row F: 10 seats available - ₹84
  Row G: 10 seats available - ₹70
